Copy this file into the same folder as the data you want to analyse.

You can run a cell by hitting  `Shift + Enter` on the keyboard

A more extensive tutorial can be found [here](https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook#gs.kwHTUSQ) Skip to: "Getting Started With Jupyter Notebooks" if you are hazy.

In [ ]:
# Import and load some packages
%pylab notebook
import sfg2d
import os

# Create some folders
if not os.path.isdir("figures"): 
    os.mkdir("figures")
if not os.path.isdir("cache"):
    os.mkdir("cache")
    
# Monkeypatch some default values
sfg2d.core.VIS_WL = 812.7
sfg2d.core.PUMP_FREQ = 2400
#sfg2d.core.BASE_SPEC = Array_of_data
#sfg2d.core.NORM_SPEC = Array_of_data

# Reload cached results 
sample = sfg2d.SfgRecord("cache/")

In [ ]:
# Automatic reloading of modules during development
%load_ext autoreload
%autoreload 2

# Backgroud

In [ ]:
# Fill in correct data path after the ./
# You can use the TAB key for autocompletion.
# Position your cursor after the ./ and hit the TAB key
kwgs = {"y_property" : "rawData", "delay_mean" : True}
bg0 = sfg2d.SfgRecord("./").subselect(**kwgs)[1].squeeze() # <- e.g. shot background scan
bg1 = sfg2d.SfgRecord("./").subselect(**kwgs)[1].squeeze()

In [ ]:
fig, ax = subplots(1, 2, figsize=(9, 4))
ax[0].plot(bg0.T)
ax[1].plot(bg1.T);

# Ir Profile

In [ ]:
q0 = sfg2d.SfgRecord("./") # Fill in path

sfg2d.replace_pixel(q0, 496) # Replace broken pixel
q0.base = bg0 # Choos correct baseline here. You can also just put a constan value.
kwgs = {'y_property': "basesubed", 'delay_mean': True}
norm_spec = q0.subselect(**kwgs)[1] # <- This will be your dataset for normalization

In [ ]:
fig, ax = subplots()
sfg2d.plotting.spec_plot(
    q0, 
    title="Quartz", 
    ylabel="Counts in {}s".format(q0.exporsure_time_s )
)
sfg2d.savefig("figures/norm") # Saves the figures as pixel and as vektor graphics.
q0.save("cache/quartz") # Saves intermediate result for easier reuse.

# Cross-Correlation

In [ ]:
cc0 = sfg2d.SfgRecord("./") # Fill in path

sfg2d.replace_pixel(cc0, 496)
cc0.base = bg0 # Use correct baseline
cc0.roi_x_pixel_spec = slice(500, 1300) # x-pixel slice of interest
cc0.rois_x_pixel_trace = [cc0.roi_x_pixel_spec]
cc0.roi_spectra = [0] # Only look at the first spectrum

xdata = cc0.pp_delays[cc0.roi_delay]
ydata = cc0.traces_basesubed.squeeze()

ydata = ydata/ydata.max()
gm = sfg2d.models.GaussianModel(xdata, ydata, p0=[400, 0, 180, 0])
gm.curve_fit()
gm.box_coords = (.1, .3)


In [ ]:
fig, ax  = subplots()
sfg2d.plotting.spec_plot(
    cc0,
    title="Pump-Probe IIV",
     ylabel="Counts in {}ms".format(cc0.exposure_time_ms)
)
sfg2d.savefig("figures/cc_spectra")

fig, ax =  subplots()
sfg2d.plotting.trace_plot(
    cc0,
    title="Cross-Correlation",
    ylabel="Mean Counts in {}ms".format(cc0.exposure_time_ms)
)
sfg2d.savefig("figures/cc_trace")

fig, ax = subplots()
gm.plot(show_fit_line=True, show_box=True)
xlabel("Time in fs")
ylabel("Intensity in a.u.")
title("Corss Correlation")
sfg2d.savefig("figures/cc")

# Pump SFG

In [ ]:
# Pump-Sfg and Probe-SFG
pump = sfg2d.SfgRecord("")

sfg2d.replace_pixel(pump, 496)
pump.base = bg0 # Set correct baseline
pump.norm = norm_spec
pump.rois_x_pixel_trace=[slice(600, 800)]
pump.roi_x_pixel_spec=pump.rois_x_pixel_trace[0]
pump.roi_spectra=[0]
pump.save("cache/pump")

xdata, ydata = pump.subselect(y_property='basesubed', x_property="wavenumber", delay_mean=True)
ydata =ydata.squeeze()

ydata = ydata/ydata.max()
gm = sfg2d.models.GaussianModel(xdata, ydata, p0=[1, 2500, 100, 0])
gm.curve_fit()
gm.box_coords = (.3, .1)

In [ ]:
fig, ax = subplots()
sfg2d.plotting.spec_plot(
    pump, 
    title="Pump SFG",
    ylabel="Counts in {}ms".format(pump.exposure_time_ms)
)
sfg2d.savefig("figures/pump")

fig, ax = subplots()
sfg2d.plotting.spec_plot(
    pump, 
    title="Pump SFG",
    ylabel="Counts in {}ms".format(pump.exposure_time_ms),
    delay_mean=True,
)
sfg2d.savefig("figures/pump_mean")

fig, ax = subplots()
gm.plot(show_fit_line=True, show_box=True)
sfg2d.savefig('figures/pump_mean_fit')

fig, ax = subplots()
sfg2d.plotting.trace_plot(
    pump,
    title="Pump-Vis Trace"
)
sfg2d.savefig("figures/pump_trace")

# Tracks

I think this is rather useless by now

In [ ]:
record = 

fig, ax = subplots()
record.plot_time_track(
    y_property="basesubed", 
    roi_spectra=[0, 2], 
    roi_x_pixel=slice(300, 1200), 
    plt_kwg={"marker": "o", "ls": 'None'}
)
title("Mean Signal vs Time")
ylabel("Counts")
xlabel("Index")
ylim(35, 50)

Plots Frame number vs signals, while pp-delays get averaged.

In [ ]:
record = 

fig, ax = subplots(figsize=(9,3))
sfg2d.plotting.frame_track(record, frame_track_kwg={'roi_spectra': [record.unpumped_index, record.pumped_index]})
tight_layout()

sfg2d.savefig("figures/{}_frame_track".format(record.metadata['material']))

# Heat Bleach

In [ ]:
hot = 
cold = 

fig, ax = subplots(2, 2, figsize=(9, 8))
sca(ax[0, 0])
title("Raw Static Spectrum")
cold.plot_spec(y_property="basesubed", x_property="wavenumber", label = "Cool")
hot.plot_spec(y_property="basesubed", x_property="wavenumber", label = "Hot")
xlabel("wavenumber in 1/cm")
legend()

sca(ax[0, 1])
title("Normalized Spectrum")
cold.plot_spec(y_property="normalized", x_property="wavenumber", medfilt_kernel=(5), marker = "o", label = "Cool")
hot.plot_spec(y_property="normalized", x_property="wavenumber", medfilt_kernel=(5), marker = "o", label = "Hot")
xlabel("wavenumber in 1/cm")
ylim(0.002, 0.012)
legend()

sca(ax[1, 0])
bleach = sfg2d.core.medfilt(median(hot.basesubed / cold.basesubed, 1)[0, 1], 5)
plot(cold.wavenumber, bleach)
title("Heat Bleach Rel")
xlim(2100, 2800)
ylim(1, 1.4)

sca(ax[1,1])
bleach = sfg2d.core.medfilt(median(hot.normalized - cold.normalized, 1)[0, 1], 5)
plot(cold.wavenumber, bleach)
title("Heat Bleach Abs")
xlim(2100, 2800)
ylim(0, 0.0025)
tight_layout()
sfg2d.savefig("figures/{}_heat_bleach".format(hot.metadata["material"]))

# Pump Probe

Loading of a pump probe data file and setting its parameters.

In [ ]:
g0 = sfg2d.SfgRecord("./")# Fill in path
record = g0

# Victor Camera has a broken Pixel 496. This replaces the values of the broken pixel.
sfg2d.replace_pixel(record, 496)
record.base = bg0 # Use correct baseline
record.norm = norm_spec
record.pumped_index = 2
record.unpumped_index = 0
record.rois_x_pixel_trace=[slice(600, 800),] # Where the pump is and you want to have traces.
record.roi_x_pixel_spec = slice(200, 1400) # Where the static signal is
record.zero_time_selec=[0, 1, 2] # Lets use the first the pp-delay points for offset correction
record.zero_time_subtraction = True # Perform zero time offset correction.
record.save("cache/{}_pump{}".format(record.metadata['material'], record.metadata['pump_freq']))

#record.save("cache/gold")

Import a list of files as a singel record

In [ ]:
record_names = (
    "", 
    "", 
    "", 
    "",
    "",
    "",
)
na2so4 = sfg2d.core.SfgRecords_from_file_list(record_names)

## Frame Track

In [ ]:
record = 

fig, ax = subplots(figsize=(9,3))
sfg2d.plotting.frame_track(record)
tight_layout()

sfg2d.savefig("figures/{}_frame_track".format(record.metadata['material']))

## Pumped and unpumped

In [ ]:
record = 

fig, ax = subplots(2, 2, squeeze=False, figsize=(9, 8), sharex=True, sharey="row")
sca(ax[0, 0])
title("Unpumped Spectra")
record.plot_spec(y_property="unpumped", x_property="wavenumber", roi_spectra=[0])
ylim(0, 100)

sca(ax[0, 1])
title("Pumped Spectra")
record.plot_spec(y_property="pumped", x_property="wavenumber",  roi_spectra=[0])


sca(ax[1, 0])
title("Unpumped Spectra")
record.plot_spec(y_property="unpumped_norm", x_property="wavenumber",  roi_spectra=[0])
xlabel("wavenumber in 1/cm")
ylim(0, 0.01)

sca(ax[1, 1])
title("Pumped Spectra")
record.plot_spec(y_property="pumped_norm", x_property="wavenumber",  roi_spectra=[0])
xlabel("wavenumber in 1/cm")

sfg2d.savefig("figures/{}_static_spectra".format(record.metadata.get('material')))

In [ ]:
record = g0

fig, ax = subplots(1, 2, figsize=(9,4), sharey=True)
sca(ax[0])
sfg2d.plotting.spec_plot(
    record,
    title="Unpumped", 
    y_property="unpumped",
    ylabel="Counts in {:.0f}ms".format(record.metadata.get('exposure_time').microseconds/1000)
)

sca(ax[1])
sfg2d.plotting.spec_plot(
    record,
    title="Pumped", 
    y_property="pumped",
    ylabel="Counts in {:.0f}ms".format(record.metadata.get('exposure_time').microseconds/1000)
)
tight_layout()

#sfg2d.savefig("figures/{}")

## Bleach

Slideable version in a notebook

In [ ]:
sfg2d.plotting.bleach_plot_slider()

The easy method

In [ ]:
record = 

attr = 'bleach_abs'
sfg2d.plotting.bleach_plot_pdf(
    record, 
    "{}_pump{}_{}".format(record.metadata['material'], record.metadata["pump_freq"], attr),
    y_property=attr)

The hard method

In [ ]:
record = g0 # Use correct SfgRecord.
attribute = "bleach_abs"
attribute = "bleach_rel"

@sfg2d.multipage_pdf # Within the decorator we hide the loop
def my_plot(index):

    record.plot_bleach(
        y_property=attribute, 
        roi_delay=[slice(index, index+1)],
        x_property="wavenumber"
    )

    title("{} @ {} fs".format(record.metadata.get("sample"), record.pp_delays[index]))
    # Usually the limits need some tweaking
    #xlim(2000, 3000) 
    ylim(0.4, 1.1)
    xlabel("wavenumber in 1/cm")

name = "figures/{}_{}_pump{}".format(
    record.metadata.get("sample"), attribute, record.metadata.get("pump_freq")
)
my_plot(name, range(record.number_of_pp_delays))

## Traces

In [ ]:
record = 
#record.rois_x_pixel_trace = [slice(900, 1100), slice(700, 900), slice(500, 700)]

fig, ax = subplots()
sfg2d.plotting.trace_plot(record, y_property="traces_bleach_rel")
xlim(-1000, 5000)
sfg2d.savefig("figures/{}_pump{}_traces".format(record.metadata["material"], record.metadata['pump_freq']))

# Combining Scans

In [ ]:
# Fill in paths to files
imp_data = (
    "./",  
    "./", 
)
data_list = []
for elm in imp_data:
    elm = sfg2d.SfgRecord(elm)
    ## Remove unneeded frames like so
    #if elm == "": # enter name of the file here
    #    sfgrecord = sfgrecord.keep_frames(slice(3)) # The second index is the frame slice
    data_list.append(elm)
record = sfg2d.concatenate_list_of_SfgRecords(data_list) # This cache the records into a single record

# Set some Parameters and Properties
record.metadata["pump_freq"] = 2500 # Mean freq of the pump
record.metadata["sample"] = "sample" # Name of the sample
record.base = bg1 # Baseline
record.norm = norm_spec # Set spectrum for normalization
record.pumped_index = 2 # Set index of pumped data
record.unpumped_index = 1 # Set index of unpumped data
record.sub_base(inplace=True) # Subtract baseline
record = record.make_static_corr() # Correct for static drifts in data
record = record.make_avg() # Make frame wise average
record.sub_base(inplace=True) # Make shure that baseline is still subtracted after corrections
record.normalize(inplace=True) # use ir profile to normalize data.
record._wavenumber = None # Reset Wavenumber for recalculation

# Apply some corrections
record = record.make_static_corr() # Correct for static drifts in data
record = record.make_avg() # Make frame wise average
record.sub_base(inplace=True) # Make shure that baseline is still subtracted after corrections
record.normalize(inplace=True) # use ir profile to normalize data.

# Save result
record.save("cache/{0}".format(record.metadata.get("sample")))

# 2D Spectrum from Scan

In [ ]:
record = 

#record.rois_x_wavenumber_trace = [slice(2309, 2449), slice(2460, 2568), slice(2578, 2661), slice(2686, 2793)]
levels = linspace(0.87, 1, 15) 
fig, ax, axl, axb, axr =sfg2d.plotting.contour(
    record, 
    pixel_med=11,
    N=50,
    levels=levels,
    show_colorbar=False,
    show_axl=True
)
axl.set_xlim(0.6, 1.1)
axb.set_xlim(-1000, 5000)

name = "figures/{}_img_pump{}".format(
    record.metadata.get("material"), record.metadata.get("pump_freq"), 
)
sfg2d.savefig(name, dpi=80)